# TD Final : Analyse des Avis et Alertes ANSSI avec Enrichissement des CVE
*Solution proposée par Laura Damas, Camille Dommergue, Maxime Cerruti et Elodie Duflaut*

Importation des libraries

In [1]:
import pandas as pd
import feedparser as fp
import requests as req
import re

Etape 1 - Extraction des Flux RSS

In [2]:
url_avis = "https://cert.ssi.gouv.fr/avis/feed"
url_alerte = "https://cert.ssi.gouv.fr/alerte/feed"

rss_feed_avis = fp.parse(url_avis)
rss_feed_alerte = fp.parse(url_alerte)

# Vérification de l'intégrité du flux
if rss_feed_avis.bozo:
    raise ValueError("Echec de l'analyse du feed RSS-AVIS. Vérifiez l'URL.\n")
elif rss_feed_alerte.bozo:
    raise ValueError("Echec de l'analyse du feed RSS-ALERTE. Vérifiez l'URL.\n")
else:
    print(f"Feed RSS-AVIS analysé avec succès (source : {url_avis})\n")
    print(f"Feed RSS-ALERTE analysé avec succès (source : {url_alerte})\n")

rows = []

def get_bulletin_id(ent_id):
    parts = ent_id.split('/')
    bulletin_string = parts[-2]
    return bulletin_string


def clean_title(title_string):
    """
    Supprime tous les textes entre crochets [] ou parenthèses () d'une chaîne de characters donnée.
    """
    cleaned_string = re.sub(r"\(.*?\)", "", title_string)
    cleaned_string = re.sub(r"\[.*?\]", "", cleaned_string)
    cleaned_string = re.sub(r"\s\s+", " ", cleaned_string).strip()

    return cleaned_string



# Collecte des données ANSSI
for entry in rss_feed_avis.entries:
    
    rows.append({
        "Id": get_bulletin_id(entry.id),
        "Title": clean_title(entry.title),
        "Type": "Avis",
        "Link": entry.link,
        "Summary": entry.summary,
        "Published": entry.published
    })

for entry in rss_feed_alerte.entries:
    
    rows.append({
        "Id": get_bulletin_id(entry.id),
        "Title": clean_title(entry.title),
        "Type": "Alerte",
        "Link": entry.link,
        "Summary": entry.summary,
        "Published": entry.published
    })

# Conversion en DataFrame
df_flux_rss = pd.DataFrame(rows, columns=["Id", "Title", "Type", "Link", "Published", "Summary"])

# Conversion de la colonne 'Publihed' en datetime
df_flux_rss["Published"] = pd.to_datetime(df_flux_rss["Published"], format="%a, %d %b %Y %H:%M:%S %z", errors='coerce')

# Tri par date de publication
df_flux_rss = df_flux_rss.sort_values(by="Published", ascending=False)

Feed RSS-AVIS analysé avec succès (source : https://cert.ssi.gouv.fr/avis/feed)

Feed RSS-ALERTE analysé avec succès (source : https://cert.ssi.gouv.fr/alerte/feed)



In [16]:
df_flux_rss

,Id,Title,Type,Link,Published,Summary,CVE
37,CERTFR-2025-AVI-0519,Multiples vulnérabilités dans Moodle,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,"[CVE-2025-49513, CVE-2025-49515, CVE-2025-4951..."
35,CERTFR-2025-AVI-0517,Multiples vulnérabilités dans les produits Veeam,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,"[CVE-2025-24287, CVE-2025-23121, CVE-2025-24286]"
38,CERTFR-2025-AVI-0520,Multiples vulnérabilités dans les produits Atl...,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,"[CVE-2025-22228, CVE-2025-31650, CVE-2024-57699]"
39,CERTFR-2025-AVI-0521,Multiples vulnérabilités dans Synacor Zimbra C...,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,[]
36,CERTFR-2025-AVI-0518,Multiples vulnérabilités dans Google Chrome,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,"[CVE-2025-6192, CVE-2025-6191]"
...,...,...,...,...,...,...,...
44,CERTFR-2023-ALE-001,Vulnérabilité dans Fortinet FortiOS,Alerte,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,2023-03-14 00:00:00+00:00,"Le 07 mars 2023, Fortinet a publié un avis de ...",[CVE-2022-41328]
43,CERTFR-2023-ALE-015,Campagne d'exploitation d'une vulnérabilité af...,Alerte,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,2023-02-03 00:00:00+00:00,\[Mise à jour du 10 février 2023\] Une nouvell...,"[CVE-2021-21974, CVE-2020-3992]"
42,CERTFR-2022-ALE-014,Multiples vulnérabilités dans AMI MegaRAC,Alerte,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,2022-12-16 00:00:00+00:00,"Le 05 décembre 2022, trois vulnérabilités resp...","[CVE-2022-40242, CVE-2022-40259, CVE-2022-2827]"
41,CERTFR-2022-ALE-008,Multiples vulnérabilités dans Microsoft Exchange,Alerte,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,2022-09-30 00:00:00+00:00,\[Mise à jour du 09 novembre 2022\] L'éditeur ...,"[CVE-2022-41040, CVE-2022-41082]"


Etape 2 - Extraction des CVE

In [5]:
def extract_cve_from_link(link, verbose=False):
    json_link = link.rstrip("/") + "/json/"
    response = req.get(json_link)
    if response.status_code != 200:
        if verbose:
            print(f"Erreur: impossible d'accéder à {json_link}")
        return []

    try:
        data = response.json()
    except ValueError:
        if verbose:
            print(f"Le contenu de {json_link} n'est pas un JSON valide.")
        return []
    
    cve_list  = [item['name'] for item in data["cves"]]

    if verbose:
        print(f"CVE trouvé: {'✅' if cve_list else '❌'}")
    
    return cve_list

In [6]:
#extract CVEs from each link in the DataFrame & store them in a new column
df_flux_rss["CVE"] = df_flux_rss["Link"].apply(extract_cve_from_link)
df_flux_rss

,Id,Title,Type,Link,Published,Summary,CVE
37,CERTFR-2025-AVI-0519,Multiples vulnérabilités dans Moodle,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,"[CVE-2025-49513, CVE-2025-49515, CVE-2025-4951..."
35,CERTFR-2025-AVI-0517,Multiples vulnérabilités dans les produits Veeam,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,"[CVE-2025-24287, CVE-2025-23121, CVE-2025-24286]"
38,CERTFR-2025-AVI-0520,Multiples vulnérabilités dans les produits Atl...,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,"[CVE-2025-22228, CVE-2025-31650, CVE-2024-57699]"
39,CERTFR-2025-AVI-0521,Multiples vulnérabilités dans Synacor Zimbra C...,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,[]
36,CERTFR-2025-AVI-0518,Multiples vulnérabilités dans Google Chrome,Avis,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,"[CVE-2025-6192, CVE-2025-6191]"
...,...,...,...,...,...,...,...
44,CERTFR-2023-ALE-001,Vulnérabilité dans Fortinet FortiOS,Alerte,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,2023-03-14 00:00:00+00:00,"Le 07 mars 2023, Fortinet a publié un avis de ...",[CVE-2022-41328]
43,CERTFR-2023-ALE-015,Campagne d'exploitation d'une vulnérabilité af...,Alerte,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,2023-02-03 00:00:00+00:00,\[Mise à jour du 10 février 2023\] Une nouvell...,"[CVE-2021-21974, CVE-2020-3992]"
42,CERTFR-2022-ALE-014,Multiples vulnérabilités dans AMI MegaRAC,Alerte,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,2022-12-16 00:00:00+00:00,"Le 05 décembre 2022, trois vulnérabilités resp...","[CVE-2022-40242, CVE-2022-40259, CVE-2022-2827]"
41,CERTFR-2022-ALE-008,Multiples vulnérabilités dans Microsoft Exchange,Alerte,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,2022-09-30 00:00:00+00:00,\[Mise à jour du 09 novembre 2022\] L'éditeur ...,"[CVE-2022-41040, CVE-2022-41082]"


Etape 3 - Enrichissement des CVE

In [10]:
# Fonction utilitaires de get_api_data

def safe_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return None


def is_actually_null(obj):
    return str(obj).lower().strip() in ['none', 'n/a', 'unspecified', 'null', '[]', '{}', '']


def find_all_by_key(json_data, the_key):
    results = []
    if isinstance(json_data, dict):
        for key, value in json_data.items():
            if key == the_key:
                results.append(value)
            else:
                results.extend(find_all_by_key(value, the_key))
    elif isinstance(json_data, list):
        for item in json_data:
            results.extend(find_all_by_key(item, the_key))
    return results


def find_first_non_null_by_key(json_data, the_key):
    if isinstance(json_data, dict):
        for key, value in json_data.items():
            if key == the_key and not is_actually_null(value):
                return value
            result = find_first_non_null_by_key(value, the_key)
            if not is_actually_null(result):
                return result
    elif isinstance(json_data, list):
        for item in json_data:
            result = find_first_non_null_by_key(item, the_key)
            if not is_actually_null(result):
                return result
    return None


def get_description(cna_adp):
    descriptions = find_first_non_null_by_key(cna_adp, "descriptions")
    if descriptions is None:
        return None
    for d in descriptions:
        if d.get("lang") == "en":
            return d.get("value")
    return None


def get_cvss_metrics(cna_adp):
    metrics = find_first_non_null_by_key(cna_adp, "metrics")
    if not metrics:
        return None, None
    for m in metrics:
        keys = [k for k in m if "cvss" in k]
        for k in keys:
            score = find_first_non_null_by_key(m[k], "baseScore")
            severity = find_first_non_null_by_key(m[k], "baseSeverity")
            if score and severity:
                return safe_float(score), severity
    return None, None


def get_cwes(cna_adp):
    pbs = find_first_non_null_by_key(cna_adp, "problemTypes")
    if not pbs:
        return []
    cwes = []
    for pb in pbs:
        desc = pb.get("descriptions")
        if desc:
            cwes += find_all_by_key(desc, "cweId")
    if cwes:
        return list(set(cwes))
    return []


def normalize_version_string(s):
    patterns = [
        (r"(below|up to|prior to|before|less than)\s+(version\s+)?([\w\.\-\_]+)", r"<= \3"),
        (r"(above|later than|after|greater than)\s+(version\s+)?([\w\.\-\_]+)", r">= \3"),
        (r"\bversion\b", r""),
    ]
    normalized = s
    for pattern, replacement in patterns:
        normalized = re.sub(pattern, replacement, normalized, flags=re.IGNORECASE)
    return normalized.strip()


def process_version(version_dict):
    processed_version = []
    version = version_dict["version"]
    if version:
        if ',' in version:
            processed_version += version.split(',')
        if version_dict.get("lessThan"):
            processed_version += [f"<= {version_dict['lessThan']}"]
        if version_dict.get("greaterThan"):
            processed_version += [f">= {version_dict['greaterThan']}"]
        if not is_actually_null(version):
            processed_version += [version]

        normalized_processed_version = [normalize_version_string(v) for v in processed_version]
        return normalized_processed_version
    return processed_version


def get_affected_products(cna_adp):
    affected = find_first_non_null_by_key(cna_adp, "affected")
    if not affected:
        return []
    prods = []
    for affected in affected:
        vendor = affected.get("vendor")
        product = affected.get("product")
        if not (is_actually_null(vendor) and is_actually_null(product)):
            versions = affected.get("versions", [])
            v_to_keep = []
            for version in versions:
                v_to_keep += process_version(version)
            prods.append({"vendor": vendor, "product": product, "versions": v_to_keep})
    return prods


def get_epss_metrics(data_dict):
    infos = next(iter(data_dict), None)
    if not infos:
        return None, None
    return safe_float(infos.get("epss")), safe_float(infos.get("percentile"))

In [11]:
def get_api_data(cve_id, verbose=None):
    description = None
    cvss_score = None
    base_severity = None
    cwes = set()
    affected_products = []
    epss_score = None
    epss_percentile = None

    # CVE API
    url = f"https://cveawg.mitre.org/api/cve/{cve_id}"
    response = req.get(url)

    if response.status_code != 200:
        if verbose:
            print(f"Erreur: impossible d'accéder à l'API CVE pour {cve_id}. Statut: {response.status_code}")
        return None
    try:
        data = response.json()
    except req.exceptions.JSONDecodeError:
        if verbose:
            print(f"Erreur: Impossible de décoder la réponse JSON de l'API CVE pour {cve_id}.")
        return None

    containers = data.get("containers", {})
    CNA = containers.get("cna", {})
    ADP = containers.get("adp", {})
    if CNA or ADP:

        # Description
        description = get_description(CNA)
        if not description:
            description = get_description(ADP)

        # CVSS score & base severity
        cvss_score, base_severity = get_cvss_metrics(CNA)
        if not (cvss_score and base_severity):
            cvss_score, base_severity = get_cvss_metrics(ADP)

        # CWE(s)
        cwes = get_cwes(CNA)
        if not cwes:
            cwes = get_cwes(ADP)

        # Produits et versions affectés
        affected_products = get_affected_products(CNA)
        if not affected_products:
            affected_products = get_affected_products(ADP)

    # EPSS API
    url = f"https://api.first.org/data/v1/epss?cve={cve_id}"
    response = req.get(url)

    if response.status_code != 200:
        if verbose:
            print(f"Erreur: impossible d'accéder à l'API EPSS pour {cve_id}. Statut: {response.status_code}")
        return None
    try:
        data = response.json()
    except req.exceptions.JSONDecodeError:
        if verbose:
            print(f"Erreur: Impossible de décoder la réponse JSON de l'API EPSS pour {cve_id}.")
        return None

    # EPSS score & percentile
    d = data.get("data")
    if d:
        epss_score, epss_percentile = get_epss_metrics(d)

    if verbose:
        print(f"Pour {cve_id}:")
        print(f"\tDescription: {'✅' if description else '❌'}")
        print(f"\tCVSS_score: {'✅' if cvss_score else '❌'}")
        print(f"\tBase_severity: {'✅' if base_severity else '❌'}")
        print(f"\tCWE: {'✅' if cwes else '❌'}")
        print(f"\tAffected_products: {'✅' if affected_products else '❌'}")
        print(f"\tEPSS_score: {'✅' if epss_score else '❌'}")
        print(f"\tEPSS_percentile: {'✅' if epss_percentile else '❌'}")

    return {
        "CVE_id": cve_id,
        "Description": description,
        "CVSS_score": cvss_score,
        "Base_severity": base_severity,
        "CWE": cwes,
        "Affected_products": affected_products,
        "EPSS_score": epss_score,
        "EPSS_percentile": epss_percentile
    }


In [12]:
# Génération d'un set de CVE uniques
set_cve_uniques = set()

cve_list = df_flux_rss["CVE"]
for cve_list in df_flux_rss["CVE"]:
    for cve_id in cve_list:
        set_cve_uniques.add(cve_id)

{'CVE-2023-53108', 'CVE-2024-58056', 'CVE-2025-32339', 'CVE-2023-53101', 'CVE-2024-9380', 'CVE-2024-8190', 'CVE-2025-37831', 'CVE-2024-45491', 'CVE-2025-22105', 'CVE-2024-56719', 'CVE-2025-21793', 'CVE-2022-49890', 'CVE-2023-6779', 'CVE-2025-21871', 'CVE-2022-49835', 'CVE-2025-30399', 'CVE-2025-22030', 'CVE-2025-21941', 'CVE-2025-21957', 'CVE-2025-37749', 'CVE-2023-48795', 'CVE-2023-7028', 'CVE-2024-26873', 'CVE-2025-22064', 'CVE-2025-22490', 'CVE-2025-3905', 'CVE-2022-49841', 'CVE-2025-21754', 'CVE-2024-8963', 'CVE-2025-21995', 'CVE-2024-29133', 'CVE-2025-22015', 'CVE-2025-21917', 'CVE-2025-21711', 'CVE-2020-3992', 'CVE-2023-4527', 'CVE-2025-21968', 'CVE-2025-33055', 'CVE-2019-20330', 'CVE-2025-22228', 'CVE-2025-21950', 'CVE-2025-4227', 'CVE-2025-5958', 'CVE-2025-21794', 'CVE-2025-22007', 'CVE-2024-11831', 'CVE-2021-21974', 'CVE-2025-49709', 'CVE-2025-21890', 'CVE-2025-43550', 'CVE-2025-21865', 'CVE-2025-30012', 'CVE-2025-5281', 'CVE-2025-37887', 'CVE-2024-53680', 'CVE-2024-13088', 'C

In [13]:
# Exraction des données via les APIs

liste_cve_info = []

for cve_id in set_cve_uniques:
    data = get_api_data(cve_id)
    if data:
        liste_cve_info.append(data)

In [14]:
df_cves = pd.DataFrame(liste_cve_info)
df_cves

,CVE_id,Description,CVSS_score,Base_severity,CWE,Affected_products,EPSS_score,EPSS_percentile
0,CVE-2023-53108,"In the Linux kernel, the following vulnerabili...",NaN,None,[],"[{'vendor': 'Linux', 'product': 'Linux', 'vers...",0.00035,0.08747
1,CVE-2024-58056,"In the Linux kernel, the following vulnerabili...",NaN,None,[],"[{'vendor': 'Linux', 'product': 'Linux', 'vers...",0.00044,0.13020
2,CVE-2023-53101,"In the Linux kernel, the following vulnerabili...",NaN,None,[],"[{'vendor': 'Linux', 'product': 'Linux', 'vers...",0.00035,0.08747
3,CVE-2024-9380,An OS command injection vulnerability in the a...,7.2,HIGH,[CWE-77],"[{'vendor': 'Ivanti', 'product': 'CSA (Cloud S...",0.82973,0.99186
4,CVE-2024-8190,An OS command injection vulnerability in Ivant...,7.2,HIGH,[CWE-78],"[{'vendor': 'Ivanti', 'product': 'CSA (Cloud S...",0.91436,0.99635
...,...,...,...,...,...,...,...,...
1059,CVE-2025-21828,"In the Linux kernel, the following vulnerabili...",NaN,None,[],"[{'vendor': 'Linux', 'product': 'Linux', 'vers...",0.00030,0.06771
1060,CVE-2023-53068,"In the Linux kernel, the following vulnerabili...",NaN,None,[],"[{'vendor': 'Linux', 'product': 'Linux', 'vers...",0.00024,0.04799
1061,CVE-2025-21728,"In the Linux kernel, the following vulnerabili...",NaN,None,[],"[{'vendor': 'Linux', 'product': 'Linux', 'vers...",0.00075,0.23498
1062,CVE-2025-21782,"In the Linux kernel, the following vulnerabili...",NaN,None,[],"[{'vendor': 'Linux', 'product': 'Linux', 'vers...",0.00028,0.06036


Etape 4 - Consolidation des Données

In [17]:
''' Current dataframes and their contents:
df_flux_rss : Id, Title, Type, Link, Published, Summary, CVE(list)
df_cves : CVE_id, Description, CVSS_score, Base_severity, CWE, Affected_products(dictionnaire), EPSS_score, EPSS_percentile

Wanted dataframe:
df_consolidated : Id, Link, Title, Type, Published, Summary, CVE_id, Description, CVSS_score, Base_severity, EPSS_score, EPSS_percentile, CWE, vendor, product, version


df_flux_rss_exploded = df_flux_rss.explode('CVE')

df_consolidated = pd.merge(df_flux_rss_exploded, df_cves, left_on="CVE", right_on="CVE_id", how="left")
# Reorder the columns in the desired order 
df_consolidated = df_consolidated[[
    "Id", "Link", "Title", "Type", "Published", "Summary",
    "CVE_id", "Description", "CVSS_score", "Base_severity",
    "CWE", "Affected_products", "EPSS_score", "EPSS_percentile"
]]

print("\nConsolidated DataFrame Info:")
print(df_consolidated.info())
print("\nConsolidated DataFrame Head:")
print(df_consolidated.head())
print("\nConsolidated DataFrame Tail:") 
print(df_consolidated.tail())'''

' Current dataframes and their contents:\ndf_flux_rss : Id, Title, Type, Link, Published, Summary, CVE(list)\ndf_cves : CVE_id, Description, CVSS_score, Base_severity, CWE, Affected_products(dictionnaire), EPSS_score, EPSS_percentile\n\nWanted dataframe:\ndf_consolidated : Id, Link, Title, Type, Published, Summary, CVE_id, Description, CVSS_score, Base_severity, EPSS_score, EPSS_percentile, CWE, vendor, product, version\n\n\ndf_flux_rss_exploded = df_flux_rss.explode(\'CVE\')\n\ndf_consolidated = pd.merge(df_flux_rss_exploded, df_cves, left_on="CVE", right_on="CVE_id", how="left")\n# Reorder the columns in the desired order \ndf_consolidated = df_consolidated[[\n    "Id", "Link", "Title", "Type", "Published", "Summary",\n    "CVE_id", "Description", "CVSS_score", "Base_severity",\n    "CWE", "Affected_products", "EPSS_score", "EPSS_percentile"\n]]\n\nprint("\nConsolidated DataFrame Info:")\nprint(df_consolidated.info())\nprint("\nConsolidated DataFrame Head:")\nprint(df_consolidated.he

In [23]:
df_flux_exploded = df_flux_rss.explode('CVE')  # une ligne par CVE

# Convertir les dictionnaires de 'Affected_products' en DataFrame à part
affected_rows = []

for _, row in df_cves.iterrows():
    cve_id = row['CVE_id']
    products = row['Affected_products']
    for prod in products:
        affected_rows.append({
            'CVE_id': cve_id,
            'vendor': prod.get('vendor'),
            'product': prod.get('product'),
            'version': prod.get('versions', [])
        })

df_products = pd.DataFrame(affected_rows)

# Exploser les DataFrames pour avoir une ligne par trucs
df_prod_exploded = df_products.explode('version')
df_cves_exploded = df_cves.explode('CWE')

# Fusionner df_cves avec les produits détaillés
df_cves_flat = pd.merge(df_cves_exploded.drop(columns=['Affected_products']), df_prod_exploded, on='CVE_id', how='left')

# Fusion finale avec les flux RSS
df_consolidated = pd.merge(
    df_flux_exploded,
    df_cves_flat,
    left_on='CVE',
    right_on='CVE_id',
    how='left'
)

# Réorganiser les colonnes
df_consolidated = df_consolidated[[
    'Id', 'Link', 'Title', 'Type', 'Published', 'Summary',
    'CVE_id', 'Description', 'CVSS_score', 'Base_severity',
    'EPSS_score', 'EPSS_percentile', 'CWE',
    'vendor', 'product', 'version'
]]


# Supprime toutes les entrées où cve_id est NaN
df_consolidated = df_consolidated.dropna(subset=['CVE_id'])

In [24]:
df_consolidated

,Id,Link,Title,Type,Published,Summary,CVE_id,Description,CVSS_score,Base_severity,EPSS_score,EPSS_percentile,CWE,vendor,product,version
7,CERTFR-2025-AVI-0519,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,Multiples vulnérabilités dans Moodle,Avis,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,CVE-2025-46337,None,10.0,CRITICAL,0.00082,0.25113,CWE-89,ADOdb,ADOdb,< 5.22.9
8,CERTFR-2025-AVI-0517,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,Multiples vulnérabilités dans les produits Veeam,Avis,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,CVE-2025-24287,A vulnerability allowing local system users to...,6.1,MEDIUM,NaN,NaN,NaN,Veeam,Backup for Microsoft Windows,6.2.0.121
9,CERTFR-2025-AVI-0517,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,Multiples vulnérabilités dans les produits Veeam,Avis,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,CVE-2025-23121,A vulnerability allowing remote code execution...,9.9,CRITICAL,NaN,NaN,NaN,Veeam,Backup and Recovery,12.3.1
10,CERTFR-2025-AVI-0517,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,Multiples vulnérabilités dans les produits Veeam,Avis,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,CVE-2025-24286,A vulnerability allowing an authenticated user...,7.2,HIGH,NaN,NaN,NaN,Veeam,Backup and Recovery,12.3
11,CERTFR-2025-AVI-0520,https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-...,Multiples vulnérabilités dans les produits Atl...,Avis,2025-06-18 00:00:00+00:00,De multiples vulnérabilités ont été découverte...,CVE-2025-22228,"BCryptPasswordEncoder.matches(CharSequence,Str...",7.4,HIGH,0.00050,0.15501,CWE-287,Spring,Spring Security,<= 5.7.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17541,CERTFR-2022-ALE-007,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,Multiples vulnérabilités dans Microsoft Windows,Alerte,2022-09-16 00:00:00+00:00,\[MàJ du 31 octobre 2022\] Une preuve de conce...,CVE-2022-34722,None,9.8,CRITICAL,0.06616,0.90721,NaN,Microsoft,Windows Server 2012 (Server Core installation),6.2.9200.0
17542,CERTFR-2022-ALE-007,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,Multiples vulnérabilités dans Microsoft Windows,Alerte,2022-09-16 00:00:00+00:00,\[MàJ du 31 octobre 2022\] Une preuve de conce...,CVE-2022-34722,None,9.8,CRITICAL,0.06616,0.90721,NaN,Microsoft,Windows Server 2012 R2,<= 6.3.9600.20571
17543,CERTFR-2022-ALE-007,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,Multiples vulnérabilités dans Microsoft Windows,Alerte,2022-09-16 00:00:00+00:00,\[MàJ du 31 octobre 2022\] Une preuve de conce...,CVE-2022-34722,None,9.8,CRITICAL,0.06616,0.90721,NaN,Microsoft,Windows Server 2012 R2,6.3.9600.0
17544,CERTFR-2022-ALE-007,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,Multiples vulnérabilités dans Microsoft Windows,Alerte,2022-09-16 00:00:00+00:00,\[MàJ du 31 octobre 2022\] Une preuve de conce...,CVE-2022-34722,None,9.8,CRITICAL,0.06616,0.90721,NaN,Microsoft,Windows Server 2012 R2 (Server Core installation),<= 6.3.9600.20571


In [25]:
# Convert dataframe to CSV
df_consolidated.to_csv("DataFrame_Complet.csv", index=False, encoding='utf-8-sig')